In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px

## Fetch some stock data
Let's go with AAPL.

In [151]:
AAPL = yf.download("AAPL",period="2y",progress=False)
AAPL

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-09-19,54.625000,54.904999,53.825001,54.592499,53.230145,108495200
2018-09-20,55.060001,55.570000,54.787498,55.007500,53.634800,106435200
2018-09-21,55.195000,55.340000,54.322498,54.415001,53.057083,384986800
2018-09-24,54.205002,55.314999,54.157501,55.197498,53.820053,110773600
2018-09-25,54.937500,55.705002,54.924999,55.547501,54.161320,98217600
...,...,...,...,...,...,...
2020-09-14,114.720001,115.930000,112.800003,115.360001,115.360001,140150100
2020-09-15,118.330002,118.830002,113.610001,115.540001,115.540001,184642000
2020-09-16,115.230003,116.000000,112.040001,112.129997,112.129997,154679000


## Visualize the stock price

In [152]:
fig = px.line(AAPL, y="Adj Close", title='AAPL Stock Price', labels = {'Adj Close':'AAPL Close Price(in USD)'})

In [153]:
fig

## Moving Average 1 (Shorter window)

Here I am choosing Exponential moving average instead of Simple Moving Average, feel free to change it to SMA instead of EMA, you can do so in the following way.
```python
ema1['Adj Close'] = AAPL['Adj Close'].ewm(span = window1).mean()

```

In [154]:
window1 = 30
sma1 = pd.DataFrame()
sma1['Adj Close'] = AAPL['Adj Close'].rolling(window = window1).mean()
sma1

,Adj Close
Date,
2018-09-19,NaN
2018-09-20,NaN
2018-09-21,NaN
2018-09-24,NaN
2018-09-25,NaN
...,...
2020-09-14,118.030282
2020-09-15,118.256901
2020-09-16,118.345646


## Moving Average 2 (Longer Window)

In [155]:
window2 = 100
sma2 = pd.DataFrame()
sma2['Adj Close'] = AAPL['Adj Close'].rolling(window = window2).mean()
sma2

,Adj Close
Date,
2018-09-19,NaN
2018-09-20,NaN
2018-09-21,NaN
2018-09-24,NaN
2018-09-25,NaN
...,...
2020-09-14,95.076405
2020-09-15,95.547320
2020-09-16,95.964374


In [156]:
fig.add_scatter(x=sma1.index,y=sma1['Adj Close'], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=sma2.index,y=sma2['Adj Close'], mode='lines',name='SMA'+str(window2))

## Combine everything

In [157]:
data = pd.DataFrame()
data['AAPL'] = AAPL['Adj Close']
data['SMA'+str(window1)] = sma1['Adj Close']
data['SMA'+str(window2)] = sma2['Adj Close']
data

,AAPL,SMA30,SMA100
Date,,,
2018-09-19,53.230145,NaN,NaN
2018-09-20,53.634800,NaN,NaN
2018-09-21,53.057083,NaN,NaN
2018-09-24,53.820053,NaN,NaN
2018-09-25,54.161320,NaN,NaN
...,...,...,...
2020-09-14,115.360001,118.030282,95.076405
2020-09-15,115.540001,118.256901,95.547320
2020-09-16,112.129997,118.345646,95.964374


## Strategy to generate buy/sell signal

In [158]:
def dualMACrossover(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1 # Flag denoting when the 2 moving averages crossed each other
    for i in range(len(data)):
        if data['SMA'+str(window1)][i] > data['SMA'+str(window2)][i]:
            if flag != 1:
                sigPriceBuy.append(data['AAPL'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA'+str(window1)][i] < data['SMA'+str(window2)][i]:
            if flag!=0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['AAPL'][i])
                flag=0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy,sigPriceSell)

In [159]:
buy_sell = dualMACrossover(data)
data['BuySignalPrice'] = buy_sell[0]
data['SellSignalPrice'] = buy_sell[1]

## Visualize the data and the strategy

In [160]:
import plotly.graph_objects as go

fig = px.line(data, y="AAPL", title='Strategy Visualization', labels = {'index':'Date'})
fig.add_scatter(x=data.index,y=data['SMA'+str(window1)], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=data.index,y=data['SMA'+str(window2)], mode='lines',name='SMA'+str(window2))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.BuySignalPrice, marker_symbol='triangle-up',
                           marker_line_color="#000000", marker_color="#000000", 
                           marker_line_width=2, marker_size=15, name='Buy'))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.SellSignalPrice, marker_symbol='triangle-down',
                           marker_line_color="#E74C3C", marker_color="#E74C3C", 
                           marker_line_width=2, marker_size=15, name='Sell'))


# fig.add_scatter(x=data.index,y=data['BuySignalPrice'], mode= 'markers',name='Buy',marker_line_color="midnightblue")

## Backtest the strategy

In [161]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

In [162]:
class DualMACrossover(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, window1)
        self.ma2 = self.I(SMA, price, window2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(AAPL, DualMACrossover,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

In [163]:
stats

Start                     2018-09-19 00:00:00
End                       2020-09-18 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                     58.9286
Equity Final [$]                      13161.2
Equity Peak [$]                       17541.9
Return [%]                            31.6116
Buy & Hold Return [%]                 95.7045
Max. Drawdown [%]                    -44.3658
Avg. Drawdown [%]                    -3.84938
Max. Drawdown Duration      219 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    2
Win Rate [%]                               50
Best Trade [%]                        34.6453
Worst Trade [%]                      -27.0828
Avg. Trade [%]                      -0.914345
Max. Trade Duration         371 days 00:00:00
Avg. Trade Duration         214 days 00:00:00
Profit Factor                         1.27924
Expectancy [%]                         30.864
SQN                               

**A return of 31% in 2 years, not bad, but there are certainly improvements that can be done, as mentioned, the aim of this notebook was to demonstrate that something as simple as this can make money in the stock market, and to mention again, do not go out and deploy this strategy without knowing what you are doing, this is purely for educational purposes, and you should consult your financial advisor before any trading decision you make, I am not responsible for any profit/loss you make using this, hopefully you enjoyed this notebook, cheers!**